# Reference
- namedtuple
https://stackoverflow.com/questions/2970608/what-are-named-tuples-in-python
```python
from collections import namedtuple
Point = namedtuple('Point', 'x y')
pt1 = Point(1.0, 5.0)
pt2 = Point(2.5, 1.5)

from math import sqrt
line_length = sqrt((pt1.x-pt2.x)**2 + (pt1.y-pt2.y)**2)
```

# stack vs cat

In [1]:
import torch

In [2]:
t1 = torch.tensor([1,1,1])
t2 = torch.tensor([2,2,2])
t3 = torch.tensor([3,3,3])

In [11]:
t1.shape

torch.Size([3])

In [13]:
t1.unsqueeze(0).shape

torch.Size([1, 3])

In [6]:
cat1 = torch.cat((t1,t2,t3), dim=0)  

In [7]:
cat1.shape   # same axis

torch.Size([9])

In [9]:
stack1 = torch.stack((t1, t2, t3), dim=0)

In [10]:
stack1.shape   # new axis

torch.Size([3, 3])

In [15]:
torch.cat(
    (
        t1.unsqueeze(0),
        t2.unsqueeze(0),
        t3.unsqueeze(0)
    ),
    dim=0
)

tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [16]:
torch.cat(
    (
        t1.unsqueeze(0),
        t2.unsqueeze(0),
        t3.unsqueeze(0)
    ),
    dim=0
).shape

torch.Size([3, 3])

In [22]:
torch.stack(
    (t1, t2, t3),
    dim=0
)

tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])

In [23]:
torch.stack(
    (t1, t2, t3),
    dim=1
)

tensor([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])

# Run builder

In [24]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [37]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple("testRun", params.keys())
        
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs

In [38]:
params = OrderedDict(
    lr = [.01, .001],
    batch_size = [1000, 10000]
)

In [39]:
print(params)

OrderedDict([('lr', [0.01, 0.001]), ('batch_size', [1000, 10000])])


In [64]:
a = []
c = []
b = namedtuple("test", params.keys())
for each in product(*params.values()):
    print(each)
    print(b(*each))

(0.01, 1000)
test(lr=0.01, batch_size=1000)
(0.01, 10000)
test(lr=0.01, batch_size=10000)
(0.001, 1000)
test(lr=0.001, batch_size=1000)
(0.001, 10000)
test(lr=0.001, batch_size=10000)


In [40]:
runs = RunBuilder.get_runs(params)
runs

[testRun(lr=0.01, batch_size=1000),
 testRun(lr=0.01, batch_size=10000),
 testRun(lr=0.001, batch_size=1000),
 testRun(lr=0.001, batch_size=10000)]

In [68]:
for i in runs:
    for j in i: 
        print(j)

0.01
1000
0.01
10000
0.001
1000
0.001
10000


In [71]:
run = runs[0]
run

testRun(lr=0.01, batch_size=1000)

In [72]:
print(run.lr)

0.01


In [74]:
for each in runs:
    print(run, run.lr, run.batch_size)

testRun(lr=0.01, batch_size=1000) 0.01 1000
testRun(lr=0.01, batch_size=1000) 0.01 1000
testRun(lr=0.01, batch_size=1000) 0.01 1000
testRun(lr=0.01, batch_size=1000) 0.01 1000


In [79]:
# another example
params_gpu_cpu = OrderedDict(
    lr = [.01, .001],
    batch_size = [1000, 10000],
    device=['cuda', 'cpu']
)

In [80]:
runs = RunBuilder.get_runs(params_gpu_cpu)
runs

[testRun(lr=0.01, batch_size=1000, device='cuda'),
 testRun(lr=0.01, batch_size=1000, device='cpu'),
 testRun(lr=0.01, batch_size=10000, device='cuda'),
 testRun(lr=0.01, batch_size=10000, device='cpu'),
 testRun(lr=0.001, batch_size=1000, device='cuda'),
 testRun(lr=0.001, batch_size=1000, device='cpu'),
 testRun(lr=0.001, batch_size=10000, device='cuda'),
 testRun(lr=0.001, batch_size=10000, device='cpu')]

### How to build the RunBuilder

In [82]:
params = OrderedDict(
    lr = [.01, .001],
    batch_size = [1000, 10000]
)

In [85]:
params.keys()

odict_keys(['lr', 'batch_size'])

In [86]:
params.values()

odict_values([[0.01, 0.001], [1000, 10000]])

In [88]:
Run = namedtuple("Run", params.keys())

In [89]:
runs = []
for v in product(*params.values()):
    runs.append(Run(*v))
runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]

In [90]:
for run in RunBuilder.get_runs(params):
    comment = f"-{run}"
    print(f"comment: {comment} lr={run.lr}, batch_size={run.batch_size}")

comment: -testRun(lr=0.01, batch_size=1000) lr=0.01, batch_size=1000
comment: -testRun(lr=0.01, batch_size=10000) lr=0.01, batch_size=10000
comment: -testRun(lr=0.001, batch_size=1000) lr=0.001, batch_size=1000
comment: -testRun(lr=0.001, batch_size=10000) lr=0.001, batch_size=10000


# CNN example with paramters tuning

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [92]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import display, clear_output

In [93]:
import pandas as pd
import time
import json

In [94]:
from itertools import product
from collections import namedtuple
from collections import OrderedDict

In [95]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=6,
                               kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, 
                               out_channels=12,
                               kernel_size=5)
        self.fc1 = nn.Linear(in_features=12*4*4,
                             out_features=120)
        self.fc2 = nn.Linear(in_features=120,
                             out_features=60)
        self.out = nn.Linear(in_features=60,
                             out_features=10)
        
    def forward(self, t):
        
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [100]:
# flatten example

t = torch.tensor([[[1, 2],
                       [3, 4]],
                      [[5, 6],
                       [7, 8]]])

In [99]:
t

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])

In [98]:
t.shape

torch.Size([2, 2, 2])

In [101]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        
        Run = namedtuple("Run", params.keys())
        
        runs = []
        
        for v in product(*params.values()):
            runs.append(Run(*v))
            
        return runs

In [103]:
class RunManager():
    def __init__(self):
        
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
        
    def begin_run(self, run, network, loader):
        
        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images', grid)
        self.tb.add_graph(
             self.network
            ,images.to(getattr(run, 'device', 'cpu'))
        )
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0   

    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0

    def end_epoch(self):
        
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
                
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
        
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self, loss, batch):
        self.epoch_loss += loss.item() * batch[0].shape[0]
        
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
    
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self, fileName):
        
        pd.DataFrame.from_dict(
            self.run_data
            ,orient='columns'
        ).to_csv(f'{fileName}.csv')
        
        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [105]:
train_set = torchvision.datasets.FashionMNIST(
    root = "./data",
    train = True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 26421880/26421880 [00:15<00:00, 1711209.49it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 98134.40it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:06<00:00, 702908.25it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 6693204.28it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [108]:
params = OrderedDict(
    lr = [.01],
    batch_size = [1000],
    shuffle=[True]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = DataLoader(train_set, batch_size=run.batch_size, shuffle=run.shuffle)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    
    m.begin_run(run, network, loader)
    for epoch in range(1):
        m.begin_epoch()
        for batch in loader:
            
            images, labels = batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights
            
            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)  
        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.019656,0.616983,5.569925,6.120687,0.01,1000,True
